In [ ]:
from cling import cling, bash

**Listing 11.1**

Caption: Parallel Fibonacci done three ways

In [ ]:
%%writefile co.cpp
#define USE_HPX 1
#ifdef USE_HPX
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <coroutine>
namespace par = hpx;
#else
#include <future>
namespace par = std;
#endif
#include <functional>
#include <iostream>

int fib1(int n) {
  if(n < 2) return n;
  par::future<int> f1 = par::async(par::launch::async,fib1,n-1);
  auto f2 = fib1(n-2);
  return f1.get() + f2;
}

#ifdef USE_HPX
par::future<int> fib2(int n) {
  if(n < 2) return par::make_ready_future(n);
  par::future<int> f1 = par::async(par::launch::async,fib2,n-1);
  auto f2 = fib2(n-2);
  return par::dataflow([](auto f1_,auto f2_){
    return f1_.get() + f2_.get();
  }, f1, f2);
}

par::future<int> fib3(int n) { 
  if(n < 2) co_return n;
  auto f1 = par::async(fib3,n-1);
  auto f2 = fib3(n-2);
  co_return (co_await f1) + (co_await f2);
}
#endif

void test_fib(int n,std::function<int(int)> f) {
  auto start_time = std::chrono::high_resolution_clock::now ();
  int r = f(n);
  auto stop_time = std::chrono::high_resolution_clock::now ();
  auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(stop_time - start_time);
  std::cout << "fib(" << n << ")=" << r << " time=" << (duration.count()*1e-9) << " secs" << std::endl;
}

int main() {
  for(int i=0;i<20;i++) {
    test_fib(i, fib1);
    #ifdef USE_HPX
    test_fib(i, [](int n)->int { return fib2(n).get(); });
    test_fib(i, [](int n)->int { return fib3(n).get(); });
    #endif
    std::cout << std::endl;
  }
  return 0;
}


In [ ]:
%%bash
hpxcxx -I . --exe=co.exe co.cpp
hpxrun.py -l 3 -t 1 ./co.exe

**Listing 11.2**

Caption: Example for the computation of the Taylor series for the natural logarithm using HPX's futures and coroutines.

In [ ]:
%%writefile taylor_corot_hpx.cpp
#include <coroutine>
#include <vector>
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <hpx/numeric.hpp>
#include <hpx/parallel/algorithm.hpp>
#include <iostream>
//-----------------------------------------------------------------


hpx::future<double> run(size_t n, size_t amount, double x) {
  std::vector<double> parts(n);
  std::iota(parts.begin(), parts.end(), 1);

  size_t partitions = std::round(n / amount);

  std::vector<hpx::future<double>> futures;
  for (size_t i = 0; i < amount; i++) {
    size_t begin = i * partitions;
    size_t end = (i + 1) * partitions;
    if (i == amount - 1) end = n;

    hpx::future<double> f = hpx::async([begin, end, x, &parts]() -> double {
      std::for_each(parts.begin() + begin, parts.begin() + end, [x](double& e) {
        e = std::pow(-1.0, e + 1) * std::pow(x, e) / (e);
      });

      return hpx::reduce(parts.begin() + begin, parts.begin() + end, 0.);
    });

    futures.push_back(std::move(f));
  }

  double result = 0;

  auto futures2 = co_await hpx::when_all(futures); 

  for (size_t i = 0; i < futures2.size(); i++) result += co_await std::move(futures2[i]);  

  co_return result; 
}

int main(){
  double x = 0.3;
  int n = 100;
  hpx::future<double> res = run(n,4,x);

  std::cout<< "Difference of Taylor and C++ result "
    << res.get() - std::log1p(x) << " after " << n
    << " iterations." << std::endl;

  return EXIT_SUCCESS;
}


In [ ]:
%%bash
hpxcxx -I . --exe=taylor_corot_hpx.exe taylor_corot_hpx.cpp
hpxrun.py -l 3 -t 1 ./taylor_corot_hpx.exe

**Listing 11.3**

Caption: Example for the computation of the Taylor series for the natural logarithm using HPX's futures and coroutines.

In [ ]:
%%writefile man_coroutine_fractal.cpp
#include <vector>
#include <pbm.hpp>
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
#include <kernel.hpp>
//-----------------------------------------------------------------

void launch(size_t start, size_t end, PBM *pbm) {

  for (size_t i = start; i < end; i ++)
    pbm->row(i) = compute_row(i);
}

hpx::future<void> run(size_t partitions, PBM *pbm) {
  std::vector<hpx::future<void>> futures;

  size_t size = size_y / partitions;

  for (size_t i = 0; i < partitions; i++) {
    size_t start = i * size;
    size_t end = (i + 1) * size;
    if (i == partitions - 1) end = size_y;
    auto f = hpx::async(launch, start, end, pbm);

    futures.push_back(std::move(f));
  }

  auto futures2 = co_await hpx::when_all(std::move(futures)); 

  co_return; 
}

int main(){

  PBM pbm(size_y, size_x);

  auto start_time = std::chrono::high_resolution_clock::now();

  run(4,&pbm).get();

  auto stop_time = std::chrono::high_resolution_clock::now();
  auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(
    stop_time - start_time);
  std::cout << "Total time(s): " << duration.count()*1e-9 << std::endl;

  // Save the image
  pbm.save("image.pbm");
}


In [ ]:
%%bash
hpxcxx -I . --exe=man_coroutine_fractal.exe man_coroutine_fractal.cpp
hpxrun.py -l 3 -t 1 ./man_coroutine_fractal.exe